In [1]:
# Initial imports
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
#hardcoded variables to be replaced by user input and environment variable calls in the future
tickers = ['MSFT','GOOG']
start_date = '2020-01-01'
end_date = '2020-12-31'
timeframe = '1D'
alpaca_api_key = 'PKURMS5ELMEHV4ZGBXTQ'
alpaca_secret_key = 'NhYNWSU4szskB7qUMl2FXPqYoQZXwL50ahf54jds'
mc_weights = [.60,.40]

In [3]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
# Get current price data for tickers
alpaca_df = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df
#Transpose dataframe for MC Simulation format
df_1 = alpaca_df[alpaca_df["symbol"]=="MSFT"].drop("symbol", axis=1)
df_2 = alpaca_df[alpaca_df["symbol"]=="GOOG"].drop("symbol", axis=1)
#df_1['pct_change'] = df_1['close'].pct_change().dropna()
#df_2['pct_change'] = df_2['close'].pct_change().dropna()
df_ticker = pd.concat([df_1, df_2], axis=1, keys=tickers)
df_ticker.head()

MSFT                                         \
                            close     high       low trade_count    open   
timestamp                                                                  
2020-01-02 05:00:00+00:00  160.62  160.730  158.3300      175506  158.78   
2020-01-03 05:00:00+00:00  158.62  159.945  158.0600      166336  158.32   
2020-01-06 05:00:00+00:00  159.03  159.100  156.5100      148395  157.08   
2020-01-07 05:00:00+00:00  157.58  159.670  157.3200      167838  159.32   
2020-01-08 05:00:00+00:00  160.09  160.800  157.9491      198632  158.93   

                                                    GOOG                      \
                             volume        vwap    close     high        low   
timestamp                                                                      
2020-01-02 05:00:00+00:00  25472631  159.837156  1367.37  1368.14  1341.5500   
2020-01-03 05:00:00+00:00  24389239  159.095118  1360.66  1372.50  1345.5436   
2020-01-06 05:00:00+00:00  24709110  158.525221  1394.21  1396.50  1350.0000   
2020-01-07 05:00:00+00:00  24503429  158.250564  1393.34  1402.99  1390.3800   
2020-01-08 05:00:00+00:00  31748417  159.644004  1404.32  1411.58  1390.8400   

                                                                      
                          trade_count     open   volume         vwap  
timestamp                                                             
2020-01-02 05:00:00+00:00       41395  1341.55  1555727  1358.592568  
2020-01-03 05:00:00+00:00       28616  1347.86  1324192  1363.116060  
2020-01-06 05:00:00+00:00       43019  1350.00  1851121  1387.386204  
2020-01-07 05:00:00+00:00       37868  1397.94  1616127  1396.288195  
2020-01-08 05:00:00+00:00       38570  1392.08  1663718  1403.612932

In [4]:
# Configuring a Monte Carlo simulation to forecast five years cumulative returns
MC_fiveyear = MCSimulation(
    portfolio_data = df_ticker,
    weights = mc_weights,
    num_simulation = 500,
    num_trading_days = 252*5
)
# Run the Monte Carlo simulation to forecast five years cumulative returns
MC_fiveyear.calc_cumulative_return()
tbl = MC_fiveyear.summarize_cumulative_return()
       
#print(tbl['mean'])

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.979560,0.966729,0.967902,0.997462,1.007099,1.021607,1.038976,0.988752,0.978483,0.995672,...,1.026157,1.013740,1.012204,1.007566,1.018760,1.008744,1.013015,1.011781,1.005828,0.988678
2,0.970110,0.947312,0.954440,0.981464,1.019422,1.046759,1.076454,1.001106,0.981628,0.994199,...,1.029141,1.029794,1.059224,1.005666,1.061921,1.005747,1.017814,1.014092,0.994403,0.964782
3,1.003514,0.946717,0.925440,0.956887,1.017619,1.027100,1.102371,0.994546,0.982029,0.986154,...,1.031872,1.053679,1.046936,0.992754,1.040641,1.037679,1.012826,1.030134,0.993110,0.979558
4,1.013095,0.976523,0.924011,0.967886,1.022410,1.024869,1.092558,1.003880,1.003790,1.004868,...,1.073555,1.063674,1.065358,1.006644,1.053551,1.041036,1.035738,1.043044,0.998099,0.958112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,2.220921,4.724018,4.709374,6.362476,1.787439,8.026535,3.899211,7.088997,3.086065,5.482950,...,4.808420,7.881361,7.518174,14.137985,4.321804,10.471854,3.678979,3.835798,5.375660,4.693652
1257,2.127609,4.695446,4.751624,6.351238,1.770933,7.831643,3.872940,7.072782,3.108012,5.503155,...,4.752345,7.867819,7.595975,13.903860,4.207688,10.337703,3.775779,3.796715,5.406094,4.666123
1258,2.116946,4.800007,4.763791,6.308900,1.746387,7.821499,3.839108,7.024029,3.064416,5.580595,...,4.698742,8.012850,7.479214,14.841795,4.319055,10.250360,3.732444,3.723473,5.322839,4.638346
1259,2.134688,5.021403,4.766399,6.053022,1.701487,8.020497,3.844849,7.202206,3.014671,5.509868,...,4.741258,8.014211,7.524185,14.926300,4.209467,9.825914,3.877189,3.558559,5.273120,4.715955


In [39]:
#Simulate Weights
import random 
numOfAssets=2
trials = 5
overall_weights=[]
total = 0
for i in range(trials):
    weights=[]
    for i in range(numOfAssets):
        weights.append(random.random())
    total = sum(weights)
    for i in range(len(weights)):
        weights[i]=round(weights[i]/total,2)
    overall_weights.append(weights)
print(overall_weights)

[[0.82, 0.18], [0.43, 0.57], [0.62, 0.38], [0.49, 0.51], [0.24, 0.76]]


In [41]:
portfolio_returns = {}
for i in overall_weights:
    # Configuring a Monte Carlo simulation to forecast five years cumulative returns
    MC_fiveyear = MCSimulation(
    portfolio_data = df_ticker,
    weights = mc_weights,
    num_simulation = 500,
    num_trading_days = 252*5
    )   
    # Run the Monte Carlo simulation to forecast five years cumulative returns
    MC_fiveyear.calc_cumulative_return()
    tbl = MC_fiveyear.summarize_cumulative_return()
    portfolio_returns[str(i)] = tbl
print(portfolio_returns)


Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

In [18]:
mc_returns = pd.DataFrame()
for i in overall_weights:
    MC_fiveyear = MCSimulation(
    portfolio_data = df_ticker,
    weights = i,
    num_simulation = 50,
    num_trading_days = 252*1
    )
    MC_fiveyear.calc_cumulative_return()
    tbl = MC_fiveyear.summarize_cumulative_return()
    pd.concat([mc_returns, tbl], axis=1, keys=i)
print(mc_returns)

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.


AttributeError: Sum of portfolio weights must equal one.